# Kryptografia - Baby Kyber

## Pierścień $\mathbb{Z}_n$

In [1]:
class Zn():
    def __init__(self,num,N):
        self.N=N
        self.num=num%self.N
        
    def __str__(self):
        return str(self.num)
        
    def __add__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(self.num+other.num,self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(self.num+other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for +: 'Zn' and '{type(other).__name__}'")
        
            
    def __radd__(self,other):
        if isinstance(other,int):
            return Zn(self.num+other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for +: '{type(other).__name__}' and 'Zn'")
    
    
    def __mul__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(self.num*other.num,self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(self.num*other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for *: 'Zn' and '{type(other).__name__}'")
        
    def __rmul__(self,other):
        if isinstance(other,int):
            return Zn(self.num*other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for *: '{type(other).__name__}' and 'Zn'")
    
    def __sub__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(self.num-other.num,self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(self.num-other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for -: 'Zn' and '{type(other).__name__}'")
        
            
    def __rsub__(self,other):
        if isinstance(other,int):
            return Zn(other-self.num,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for -: '{type(other).__name__}' and 'Zn'")
            
    def __pow__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(pow(self.num,other.num,self.N),self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(pow(self.num,other,self.N),self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for **: 'Zn' and '{type(other).__name__}'")
       
            
    def __rpow__(self,other):
        if isinstance(other,int):
            return Zn(pow(other,self.num,self.N),self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for **: '{type(other).__name__}' and 'Zn'")
            
    def __repr__(self):
        return str(self.num)

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [2]:
import numpy as np

class Rq():
    q=17
    f=np.array([1,0,0,0,1])
    def __init__(self,vec):
        _,r=np.polydiv(np.array(vec),self.f)
        self.vec=r%17
        
    def __repr__(self):
        return str(self.vec)
        
    def __add__(self,other):
        if isinstance(other,Rq):
            return Rq(np.polyadd(self.vec,other.vec))
        else:
            raise TypeError(f"unsupported operand type(s) for +: 'Rq' and '{type(other).__name__}'")
            
    def __sub__(self,other):
        if isinstance(other,Rq):
            return Rq(np.polyadd(self.vec,-1*other.vec))
        else:
            raise TypeError(f"unsupported operand type(s) for -: 'Rq' and '{type(other).__name__}'")
    
    
    def __mul__(self,other):
        if isinstance(other,Rq):
            return Rq(np.polymul(self.vec,other.vec))
        elif isinstance(other,int):
            return Rq(other*self.vec)
        else:
            raise TypeError(f"unsupported operand type(s) for *: 'Rq' and '{type(other).__name__}'")

        
    __rmul__=__mul__

## Baby Kyber

### Generowanie klucza

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [40]:
import random
A00 = Rq([6, 16, 16, 11])
A01 = Rq ([9, 4, 6, 3])
A10 = Rq([5, 3, 10, 1])
A11 = Rq([6, 1, 9, 15])
A = np.array([[ A00, A01], 
     [ A10, A11]])
s = np.array([Rq([-1,-1, 1, 0]),Rq([-1, 0, 1, 0])])
e = np.array([Rq([0, 1, 0, 0]),Rq([0, 1, -1, 0])])


def generate_key(A, s, e):
    As = A@s
    return As + e

t = generate_key(A,s,e)

### Szyfrowanie

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [43]:
m = np.array([Rq([1, 0, 1, 1])])
r = np.array([Rq([-1,1,0,0]), Rq([1,1,0,-1])])
e1 = np.array([Rq([0,1,1,0]),Rq([0,1,0,0])])
e2 = np.array([Rq([-1,-1,0,0])])
q = 17

def cipher (A, t, m, r, e1, e2, q):
    u = np.transpose(A) * r + e1
   # print(u)
    v = np.transpose(t) * r + e2 + (q//2 + 1) * m
    return u,v

u, v = cipher(A, t, m, r, e1, e2, q)

### Deszyfrowanie
$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$


In [8]:
def decipher (v, s, u):
    return v - np.transpose(s) * u

print(decipher(v,s,u))

[[[10. 13. 15.  3.] [13.  0.  0. 15.]]
 [[16.  5.  3.  4.] [16.  9.  6. 15.]]]
